# Multiclass Multilabel prediction For stack overflow Questions

## Import Libraries

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
import numpy as np
import pandas as pd

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from nltk import word_tokenize
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, BatchNormalization, GRU ,concatenate
from keras.models import Model
from sklearn.utils import class_weight
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras import models
from keras.models import load_model
from sklearn.metrics import roc_auc_score, accuracy_score

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Load Questions Dataset

In [7]:
questions_data = pd.read_csv('/content/gdrive/My Drive/Deep_Learning/stacksample/Questions.csv',encoding='iso-8859-1')
questions_data.head()

,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body
0,80,26.0,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...
1,90,58.0,2008-08-01T14:41:24Z,2012-12-26T03:45:49Z,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...
2,120,83.0,2008-08-01T15:50:08Z,NaN,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...
3,180,2089740.0,2008-08-01T18:42:19Z,NaN,53,Function for creating color wheels,<p>This is something I've pseudo-solved many t...
4,260,91.0,2008-08-01T23:22:08Z,NaN,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...


## Cleaning and preprocessing of Questions Data

In [8]:
questions_data.shape

(1264216, 7)

In [9]:
questions_data[questions_data.Score > 2].shape

(188015, 7)

In [10]:
questions_data = questions_data[questions_data.Score > 2]
print(questions_data.shape)
questions_data.head()

(188015, 7)


,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body
0,80,26.0,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...
1,90,58.0,2008-08-01T14:41:24Z,2012-12-26T03:45:49Z,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...
2,120,83.0,2008-08-01T15:50:08Z,NaN,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...
3,180,2089740.0,2008-08-01T18:42:19Z,NaN,53,Function for creating color wheels,<p>This is something I've pseudo-solved many t...
4,260,91.0,2008-08-01T23:22:08Z,NaN,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...


In [11]:
questions_data.drop(columns=['OwnerUserId', 'CreationDate', 'ClosedDate', 'Score'], inplace=True)
questions_data.head()

,Id,Title,Body
0,80,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...
1,90,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...
2,120,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...
3,180,Function for creating color wheels,<p>This is something I've pseudo-solved many t...
4,260,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...


In [12]:
questions_data['Text'] = questions_data[questions_data.columns[1:]].apply(
    lambda x: ' '.join(x.dropna().astype(str)),
    axis=1
)
questions_data.head()

,Id,Title,Body,Text
0,80,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...,SQLStatement.execute() - multiple queries in o...
1,90,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...,Good branching and merging tutorials for Torto...
2,120,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...,ASP.NET Site Maps <p>Has anyone got experience...
3,180,Function for creating color wheels,<p>This is something I've pseudo-solved many t...,Function for creating color wheels <p>This is ...
4,260,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...,Adding scripting functionality to .NET applica...


In [ ]:
!pip install beautifulsoup4

In [14]:
from bs4 import BeautifulSoup

questions_data['Text'] = questions_data['Text'].apply(lambda x: BeautifulSoup(x).get_text())
questions_data['Text'].replace('[^a-zA-Z]',' ', regex=True, inplace=True)
questions_data['Text'] = questions_data['Text'].str.lower()
questions_data['Text'] = questions_data['Text'].replace('\s+', ' ', regex=True)

In [15]:
stop_words = set(stopwords.words('english')) 
len(stop_words)
questions_data['Text'] = questions_data['Text'].apply(lambda x: ' '.join(term for term in x.split() if term not in stop_words))

In [16]:
lemmatizer = WordNetLemmatizer()
questions_data['Text'] = questions_data['Text'].apply(lambda x: ' '.join(lemmatizer.lemmatize(term) for term in x.split()))

In [17]:
questions_data.head()

,Id,Title,Body,Text
0,80,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...,sqlstatement execute multiple query one statem...
1,90,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...,good branching merging tutorial tortoisesvn re...
2,120,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...,asp net site map anyone got experience creatin...
3,180,Function for creating color wheels,<p>This is something I've pseudo-solved many t...,function creating color wheel something pseudo...
4,260,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...,adding scripting functionality net application...


In [18]:
questions_data.drop(columns=['Title', 'Body'], inplace=True)
questions_data.head()

,Id,Text
0,80,sqlstatement execute multiple query one statem...
1,90,good branching merging tutorial tortoisesvn re...
2,120,asp net site map anyone got experience creatin...
3,180,function creating color wheel something pseudo...
4,260,adding scripting functionality net application...


## Load Tags Dataset

In [19]:
tags_data = pd.read_csv('/content/gdrive/My Drive/Deep_Learning/stacksample/Tags.csv',encoding='iso-8859-1')
tags_data.head()

,Id,Tag
0,80,flex
1,80,actionscript-3
2,80,air
3,90,svn
4,90,tortoisesvn


## Get Top 10 Tags and Prepare Data

In [20]:
tags_data.shape

(3750994, 2)

In [21]:
top10 = list(tags_data.Tag.value_counts()[:10].index)
top10

['javascript',
 'java',
 'c#',
 'php',
 'android',
 'jquery',
 'python',
 'html',
 'c++',
 'ios']

In [22]:
tags_top10 = tags_data[tags_data.Tag.isin(top10)]
print (tags_top10.shape)
tags_top10.head()

(826739, 2)


,Id,Tag
14,260,c#
18,330,c++
28,650,c#
35,930,c#
39,1010,c#


In [23]:
def add_tags(question_id):
    return tags_top10[tags_top10['Id'] == question_id['Id']].Tag.values

top10 = tags_top10.apply(add_tags, axis=1)
len(top10),tags_top10.shape

(826739, (826739, 2))

In [24]:
top10.head()

14     [c#]
18    [c++]
28     [c#]
35     [c#]
39     [c#]
dtype: object

In [25]:
tags_top10.head()

,Id,Tag
14,260,c#
18,330,c++
28,650,c#
35,930,c#
39,1010,c#


In [26]:
tags_top10 = pd.concat([tags_top10, top10.rename('Tags')], axis=1)
tags_top10.head()

,Id,Tag,Tags
14,260,c#,[c#]
18,330,c++,[c++]
28,650,c#,[c#]
35,930,c#,[c#]
39,1010,c#,[c#]


In [27]:
tags_top10.drop(["Tag"], axis=1, inplace=True)
tags_top10.shape

(826739, 2)

In [28]:
top10_tags = tags_top10.loc[tags_top10.astype(str).drop_duplicates().index]
top10_tags.shape

(706336, 2)

In [30]:
top10_tags.head()

,Id,Tags
14,260,[c#]
18,330,[c++]
28,650,[c#]
35,930,[c#]
39,1010,[c#]


## Merge Questions data and Tags Data

In [31]:
all_data = pd.merge(questions_data, top10_tags, on='Id')
print(all_data.shape)
all_data.head()

(102882, 3)


,Id,Text,Tags
0,260,adding scripting functionality net application...,[c#]
1,330,use nested class case working collection class...,[c++]
2,650,automatically update version number would like...,[c#]
3,930,connect database loop recordset c simplest way...,[c#]
4,1010,get value built encoded viewstate need grab ba...,[c#]


In [32]:
multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(all_data.Tags)
labels = multilabel_binarizer.classes_
labels

array(['android', 'c#', 'c++', 'html', 'ios', 'java', 'javascript',
       'jquery', 'php', 'python'], dtype=object)

In [28]:
all_data.shape

(102882, 3)

## Split into Train and Test Set

In [33]:
train,test = train_test_split(all_data,test_size=0.2,random_state=21)

In [34]:
train.shape, test.shape

((82305, 3), (20577, 3))

In [35]:
x_train = train['Text']
y_train = multilabel_binarizer.transform(train['Tags'])
x_test = test['Text']
y_test = multilabel_binarizer.transform(test['Tags'])

In [36]:
sent_lens=[]
for sent in train['Text']:
    sent_lens.append(len(word_tokenize(sent)))
max(sent_lens)

3606

In [37]:
np.quantile(sent_lens,0.98)

508.0

In [38]:
max_len = 508
tok = Tokenizer(char_level=False,split=' ')
tok.fit_on_texts(x_train)
sequences_train = tok.texts_to_sequences(x_train)

In [39]:
vocab_len = len(tok.index_word.keys())
vocab_len

245003

In [40]:
sequences_matrix_train = sequence.pad_sequences(sequences_train, maxlen=max_len)
sequences_matrix_train

array([[     0,      0,      0, ...,     15,    686,    116],
       [     0,      0,      0, ...,     36,   1286,    105],
       [    87,    303,    921, ...,   2592,     67, 144545],
       ...,
       [     0,      0,      0, ...,     12,   1859,    780],
       [     0,      0,      0, ...,     13,    116,    377],
       [     0,      0,      0, ...,  38478,  38478,    130]], dtype=int32)

In [41]:
sequences_test = tok.texts_to_sequences(x_test)
sequences_matrix_test = sequence.pad_sequences(sequences_test, maxlen=max_len)

In [42]:
sequences_matrix_train.shape, sequences_matrix_test.shape, y_train.shape, y_test.shape

((82305, 508), (20577, 508), (82305, 10), (20577, 10))

## Build the Model

In [39]:

inputs = Input(name='inputs', shape=[max_len])
layer = Embedding(input_dim = vocab_len+1, output_dim = 500, input_length = max_len, mask_zero=True)(inputs)
layer = LSTM(64)(layer) 
layer = Dropout(0.2)(layer)
layer = Dense(256,activation='relu')(layer)
layer = Dropout(0.2)(layer)
layer = Dense(128,activation='relu')(layer)
layer = Dropout(0.2)(layer)
layer = Dense(10,activation='sigmoid')(layer)
model = Model(inputs=inputs, outputs=layer)


In [40]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 508)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 508, 500)          122502000 
_________________________________________________________________
lstm (LSTM)                  (None, 64)                144640    
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 256)               16640     
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896 

In [41]:
filepath='/content/gdrive/My Drive/Deep_Learning/stacksample/weights-{epoch:02d}-{val_loss:.4f}.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, 
                             save_best_only=True)

In [42]:
earlystop = EarlyStopping(monitor='val_loss', patience=3, verbose=1)

In [43]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

## Train the Model

In [44]:
model.fit(sequences_matrix_train,y_train,
              epochs=20,
              batch_size=128,
              validation_data=(sequences_matrix_test,y_test),
              callbacks = [earlystop, checkpoint])

Epoch 1/20
644/644 [==============================] - 1491s 2s/step - loss: 0.3313 - accuracy: 0.2867 - val_loss: 0.1708 - val_accuracy: 0.6842

Epoch 00001: val_loss improved from inf to 0.17084, saving model to /content/gdrive/My Drive/Deep_Learning/stacksample/weights-01-0.1708.h5
Epoch 2/20
644/644 [==============================] - 1478s 2s/step - loss: 0.1412 - accuracy: 0.7455 - val_loss: 0.1715 - val_accuracy: 0.7466

Epoch 00002: val_loss did not improve from 0.17084
Epoch 3/20
644/644 [==============================] - 1476s 2s/step - loss: 0.1108 - accuracy: 0.8189 - val_loss: 0.0974 - val_accuracy: 0.8270

Epoch 00003: val_loss improved from 0.17084 to 0.09743, saving model to /content/gdrive/My Drive/Deep_Learning/stacksample/weights-03-0.0974.h5
Epoch 4/20
644/644 [==============================] - 1477s 2s/step - loss: 0.0761 - accuracy: 0.8769 - val_loss: 0.1020 - val_accuracy: 0.8237

Epoch 00004: val_loss did not improve from 0.09743
Epoch 5/20
644/644 [==============

In [43]:
best_model = load_model('/content/gdrive/My Drive/Deep_Learning/stacksample/weights-05-0.0963.h5')

In [44]:
best_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 508)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 508, 500)          122502000 
_________________________________________________________________
lstm (LSTM)                  (None, 64)                144640    
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 256)               16640     
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896 

## Test the Model

In [45]:
predictions = best_model.predict(sequences_matrix_test)

## Evaluate with ROC-AUC Score

In [46]:
roc_auc_score(y_test,predictions)

0.9768599866754253

## Make a Prediction

In [47]:
def pred_tags(index):
    pred = predictions[index].round(decimals = 2)
    threshold = 0.3
    pred[pred>=threshold]=1
    pred[pred<threshold]=0
    pred = pred.astype(int)
    cnt=0
    pred_label = []
    for i in pred:
      if i==1:
          pred_label.append(labels[cnt])
      cnt = cnt+1  

    print("Predicted Tags - ", pred_label)
    

In [48]:
index = 216
print("Original Text and Tags - ", test.iloc[index])
pred_tags(index)

Original Text and Tags -  Id                                                4334600
Text    changing background color ajax jquery scenario...
Tags                            [php, javascript, jquery]
Name: 18848, dtype: object
Predicted Tags -  ['javascript', 'jquery', 'php']


In [49]:
index = 10167
print("Original Text and Tags - ", test.iloc[index])
pred_tags(index)

Original Text and Tags -  Id                                               36780200
Text    make struct immutable stack overflow internet ...
Tags                                                 [c#]
Name: 99637, dtype: object
Predicted Tags -  ['c#']


In [53]:
index = 18675
print("Original Text and Tags - ", test.iloc[index])
pred_tags(index)

Original Text and Tags -  Id                                               35910550
Text    problem installing java android studio popup t...
Tags                                      [java, android]
Name: 98712, dtype: object
Predicted Tags -  ['android', 'java']


## Save Model

In [50]:
best_model.save('/content/gdrive/My Drive/Deep_Learning/stacksample/stack_overflow_tag_prediction_model.h5')